# Configuração do Ambiente

In [1]:
# Importa o módulo de funções
%run funcoes.ipynb
from tqdm.notebook import tqdm_notebook as tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.6 MB/s eta 0:00:00


# Funções

In [2]:
def get_registros_from_html(path:str, nome_arq='FolhaPagamento') -> pd.DataFrame:
  '''
  Funciona para folhas de pagamento de empresas quem tem o sistema FORTES :
  CERUS, FIBRA, GA, GESTART, HISEG, PATRIMONIAIS, PRIMEE, UNICA e VIGON.
  Retorna um DataFrame contendo todos os registros da folha de pagamento que está no diretório
  passado no parãmetro 'path'. Sendo um registro uma linha contendo os seguintes campos:
  'Competência', 'Empregador', 'CNPJ do Empregador', 'Código do Empregador', 'Nome do Empregado', 'Cargo',
  'Admissão', 'Lotação', 'Codigo do Evento', 'Descrição do Evento', 'Total do Provento', 'Total do Desconto'.
  Haverá um registro para cada evento em sua Folha de pagamento.
  '''
  # Abre o arquivo e remove informações desnecessárias do arquivo
  arq_html = clean_html(path)

  # Declaração de variáveis
  columns = [
      'Competência','Empregador', 'CNPJ', 'Código_Empregado','Nome_Empregado',
      'Cargo', 'Admissão','Lotação','Código_Evento','Desc_Evento','Provento','Desconto']
  registros = pd.DataFrame(columns=columns)

  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  admissao = str()
  lotacao = str()
  cargo = str()
  cnpj = str()

  # Encontra o CNPJ e Empregador que são únicos para o arquivo
  for i, row in enumerate(arq_html):
    if row ==  'Empregador':
      # O empregador está uma linha após a palavra
      empregador = arq_html[i+1]
      cnpj = str(arq_html[i-1])
      # Remove alguns itens do CNPJ para padronização
      remover = ['CNPJ: ', '.', '/', '-']
      for item in remover:
        cnpj = cnpj.replace(item, '')
      break

  # Total de empregados que serão coletados:
  total_func = 0
  for row in arq_html:
    if row == 'Data e Assinatura':
      total_func += 1

  # A variável start define o começo dos dados de cada empregado
  start = 0

  for _ in tqdm(range(total_func), leave=False):

    # A coleta destas informações não é feita por uso de um 'for'
    # pois a depender da empresa, alguma informação pode mudar
    # de posição

    # Encontra a competência
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Competência':
        # A competência está uma linha após esta palavra
        competencia = arq_html[i+1]
        break
    # Encontra a lotação
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Lotação':
        # A lotação está uma linha após a palavra
        lotacao = arq_html[i+1]
        break
    # Encontra o cargo
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Cargo':
        # O cargo está uma linha após a palavra
        cargo = arq_html[i+1]
        break
    # Encontra a admissão
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Admissão':
        # A admissão está uma linha após a palavra
        admissao = arq_html[i+1]
        break
    # Encontra o código e nome do empregado
    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Empregado':
        # O código e nome estão uma linha após a palavra
        cod_e_nome = arq_html[i+1].split()
        codigo_emp = cod_e_nome[0]
        nome_emp = ' '.join(cod_e_nome[1:])
        break

    # Guarda os eventos do funcionário
    index = 0
    eventos = list()
    registro = list()

    for i, row in enumerate(arq_html[start:], start=start):
      # Os proventos começam após a linha com 'Cod.'
      if row == 'Cod.':
        index = i+1
        break


    while arq_html[index] != 'Total de Proventos':
      registro.append(arq_html[index])
      index += 1

      if len(registro) == 5:
        # Verifica se o código é não nulo
        if arq_html[index-1] or arq_html[index]:
          eventos.append(registro)
          registro = list()
        # Caso seja nulo, os eventos acabaram
        else:
          break

    # Converte a lista de eventos para DataFrame
    eventos = pd.DataFrame(eventos)
    # Pode acontecer de aparecer um funcionário sem registros
    # caso esteja afastado, então será ignorado
    if not eventos.empty:
      # Larga a coluna com as referências
      eventos = eventos.drop(2, axis='columns')
      # Inverte a ordem as colunas
      eventos = eventos[eventos.columns[::-1]]
      # Atualiza o nome das colunas
      eventos.columns = ['Código', 'Descrição', 'Proventos', 'Descontos']
    else:
      registro = [competencia, empregador, cnpj, codigo_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([999, 'Afastado', 0, 0])

      index = len(registros)
      registros.loc[index] = registro


    for i in range(eventos.shape[0]):
      # Cria uma lista com todos os dados do registro
      registro = [competencia, empregador, cnpj, codigo_emp, nome_emp, cargo, admissao, lotacao]
      registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Proventos'].iloc[i], eventos['Descontos'].iloc[i]])
      # Adiciona uma nova linha na tabela de registros
      index = len(registros)
      registros.loc[index] = registro


    # Encontra os dados do próximo funcionário
    for i, row in enumerate(arq_html[start+1:], start=start+1):
      if row == 'Data e Assinatura':
        start = i
        #if not nome_emp in arq_html[i+18]:
        break

  registros = padroniza(registros)

  # Salva a tabela como xlsx
  nome_arq = f'{nome_arq}.xlsx'
  #registros.to_excel(nome_arq, index=False)

  return registros

# Leitura dos Arquivos

In [3]:
# Lista todos os arquivos html no diretório
files = [file for file in os.listdir() if '.html' in file]
folha = pd.DataFrame()
# Lê cada arquivo no diretório e junta todos em um único DataFrame
for file in tqdm(files):
  folha_temp = get_registros_from_html(file)
  folha = pd.concat([folha, folha_temp])
# Converte o DataFrame para excel
folha.to_excel('Folha_CONDAP_91.xlsx', index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
clear()

In [ ]:
1+2

3